## 1. Download a timeseries of (1) daily deaths, (2) confirmed cases, (3) recovered per country (from John Hopkins Covid-19 Data Github repository)

In [59]:
import pandas as pd
import numpy as np
import bokeh

# Download daily deaths per country from Github repo raw csv file and save to Pandas dataframe
url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
url2 = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
url3 = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv'

deaths_df = [pd.read_csv(url, error_bad_lines=False),'deaths']
confirmed_df = [pd.read_csv(url2, error_bad_lines=False), 'confirmed']
recovered_df = [pd.read_csv(url3, error_bad_lines=False), 'recovered']

list_of_dataframes = [deaths_df, confirmed_df, recovered_df]


## 2. Create functions to unpivot and prepare data for the 3 dataframes 

### 2. (a) Aggregate state data for countries which have negligible data & unpivot

In [60]:
# Extract df of Countries without state info - remove China, Aus and Canada

def agg_countries_and_unpivot(dataframe):

    list_countries_with_state_info = ['China','Australia','Canada']

    df_countries_without_state_info = dataframe[~dataframe['Country/Region'].isin(list_countries_with_state_info)]

    # Aggregate (squash) data of countries with negligible state Info (A)
    df_countries_without_state_info = df_countries_without_state_info.groupby('Country/Region').sum()
    df_countries_without_state_info = df_countries_without_state_info.reset_index()
    df_countries_without_state_info.insert(1, 'Province/State',np.nan)

    # Get dataframe of Countries with State Info (B)
    df_countries_with_state_info = dataframe[dataframe['Country/Region'].isin(list_countries_with_state_info)]

    # Append aggregated Countries (A) with (B) dataframes
    combined_df = df_countries_with_state_info.append(df_countries_without_state_info)
    combined_df = combined_df.drop(["Lat", "Long"], axis=1)

    # Get date headers
    date_headers = list(combined_df.columns[2:].values)

    long_df = pd.melt(combined_df, id_vars= ['Country/Region','Province/State'], value_vars= date_headers)
    return long_df

### 2. (b) Rename columns & add change column

In [61]:
def rename_add_change_column(dataframe, dict_key):
    header_total_dictionary = {'deaths': 'total_deaths', 'confirmed': 'total_confirmed', 'recovered': 'total_recovered' }
    header_change_dictionary = {'deaths': 'deaths_change', 'confirmed': 'confirmed_change','recovered': 'recovered_change' }

    # Rename Columns
    renamed_df = dataframe.rename(columns={"Country/Region": "country", "Province/State": "state","variable": "date", "value": header_total_dictionary[dict_key]}, errors="raise")

    # Change date column type
    renamed_df['date'] = pd.to_datetime(renamed_df['date'])

    # Include change column
    renamed_df.insert(4, header_change_dictionary[dict_key],0)
    return renamed_df


### 2. (c) Calculate daily change for each country (and for state, if applicable)

In [62]:
# Calculate xxx_change (deaths, confirmed, recovered) for each country in 'countries' list

# Get list of unique country names
def calc_daily_change_column(dataframe, dict_key):
    header_total_dictionary = {'deaths': 'total_deaths', 'confirmed': 'total_confirmed', 'recovered': 'total_recovered' }
    
    copy_of_dataframe = dataframe
    
    full_country_list = list(copy_of_dataframe['country'].unique())

    for country in full_country_list:

        if ( country not in list_countries_with_state_info ): 

            # Set temporary df for country
            temp_df = copy_of_dataframe.loc[copy_of_dataframe['country'] == country]

            # Find difference between rows (returns difference results dataframe)
            diff = temp_df[header_total_dictionary[dict_key]].diff()

            # Apply difference calculation to original copy_of_dataframe according to index
            copy_of_dataframe.iloc[diff.index,4] = diff

        else: 
            # Deal with Australia, China and Canada (purely state data)
            # Set temporary df for country
            temp_country_df = copy_of_dataframe.loc[copy_of_dataframe['country'] == country]

            # Get unique list of states
            states_list = list(temp_country_df['state'].unique())

            for state in states_list:
                temp_state_df = temp_country_df.loc[copy_of_dataframe['state'] == state]

                diff = temp_state_df[header_total_dictionary[dict_key]].diff()

                copy_of_dataframe.iloc[diff.index,4] = diff 

    # Remove NaN values from dataframe
    copy_of_dataframe = copy_of_dataframe.fillna(0)
    
    return copy_of_dataframe

## 3. Run calculations for all 3 dataframes with defined functions

In [63]:
# Run calculations for all 3 dataframes with defined functions

for dataframe in list_of_dataframes:
    unpivoted_df = agg_countries_and_unpivot(dataframe[0])
    renamed_unpivoted_df = rename_add_change_column(unpivoted_df, dataframe[1])
    calculated_df = calc_daily_change_column(renamed_unpivoted_df, dataframe[1])
    if dataframe[1] == 'deaths':
        calculated_deaths_df = calculated_df
    elif dataframe[1] == 'confirmed':
        calculated_confirmed_df = calculated_df
    elif dataframe[1] == 'recovered':
        calculated_recovered_df = calculated_df
        

## 3. Merge all 3 calculated dataframes

In [64]:
# Join deaths, confirmed & recovered results
interim_result = pd.merge(calculated_deaths_df, calculated_confirmed_df, on=['country', 'state','date'])
final_result = pd.merge(interim_result, calculated_recovered_df, on=['country', 'state','date'])

## 4. Write to CSV

In [65]:
# Write daily deaths dataframe to CSV file (for reference)
final_result.to_csv('time_series_covid19_deaths_confirmed_recovered_global.csv', index = False, header=True)

## 5. Write to Google Sheets

In [30]:
#Some useful functions
def check_latest_country_data(dataframe, country):
    return dataframe[dataframe['country']==country].sort_values(by='date',ascending=False)[0:50]
